In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import ksource as ks
import mcpl

# Create KSource

### Input cell:
It is recommiended to complete template in a duplicated cell

In [ ]:
# Open particle list

plfile = "plistfilename"
readformat = "mcpl" # "mcpl", "ssw", "phits", "ptrac", "stock"
trasl = None
rot = None
x2z = False
plist = ks.PList(plfile, readformat, trasl=trasl, rot=rot, switch_x2z=x2z)

# Create Geometry

trasl = None
rot = None
metric = ks.GeomFlat(trasl=trasl, rot=rot)

# Create KSource

J = 1 # Source intensity [1/s]
s = ks.KSource(plist, metric, J=J)

# Evaluate statistics

In [ ]:
# With MCPL

file = s.plist.filename
mcpl.dump_stats(mcpl.collect_stats(file))
mcpl.plot_stats(file)

In [ ]:
# With ks.Stats

# Select variables ranges (box)
dxymax = -1
dzmin = np.sqrt(1-dxymax**2)
maskfun = ks.Box([None,None,None,None,None,None,dzmin], None)

# Weighting function
fact_dosim = ks.H10(pt=plist.pt)
weightfun = lambda parts: fact_dosim(parts[:,0])

parts,ws = plist.get()
stats = ks.Stats(parts, ws, weightfun=weightfun, maskfun=maskfun)

N,I,err = stats.mean_weight(steps=100)
plt.show()
N,mn,err = stats.mean(var=1, steps=100)
plt.show()
N,mn,err = stats.std(var=1, steps=100)
plt.show()

# Optimize BW

It is recommended to execute last 2 methods with N=-1 (use all
particles for fit) and save source.

Then you can optimize again with lower N, so that plots do not take too long.

In [ ]:
# Silverman method
# Fast but not very precise. Useful for rough estimation

s.bw_method = "silv"
N = 1E4
s.fit(N)

In [ ]:
# Max Likelihood Cross Validation method
# Robust method. Use previous Silverman BW as seed

s.bw_method = "mlcv"
N = 1E4
s.fit(N, seed=s.kde.bw)

In [ ]:
# K Nearest Neighbours method
# Adaptive BW. Depends on the chosen K

s.bw_method = "knn"
N = -1
s.fit(N, K=10)

print("Mean bw:")
print(s.kde.bw.mean(axis=0))

In [ ]:
# Max Likelihood Cross Validation method
# Robust method. Use previous KNN adaptive BW as seed

# Run CV with subset of particle list
s.bw_method = "mlcv"
N = 1E4
bw = s.kde.bw
s.fit(N, seed=bw[:int(N)])

# Apply CV factor to all BW
bw *= s.kde.bw[0]/bw[0]
s.kde.bw_method = bw # Store full BW in KDE
s.bw_method = None
s.fit(len(bw)) # Store full particle list in KDE

# Save KSource

In [ ]:
xmlfilename = None # XML source parameters file name
bwfilename = None  # BW file name. Only for adaptive BW

xmlfilename = s.save(xmlfilename, bwfilename)

# Create plots

In [ ]:
# Energy plots

vec0 = None
vec1 = None
EE = np.logspace(-9,0,50)
fig,scores = s.plot_E(EE, vec0, vec1)
plt.show()

In [ ]:
# XY plots

vec0 = None
vec1 = None
xx = np.linspace(-30,30,30)
yy = np.linspace(-30,30,30)
fig,[scores,errs] = s.plot2D_integr(["x","y"], [xx,yy], vec0, vec1)
plt.show()

In [ ]:
# ZT plots (guide)

Lz = s.geom.ms[1].zmax
Lt = 2 * (s.geom.ms[1].xwidth + s.geom.ms[1].yheight)

vec0 = None
vec1 = None 
zz = np.linspace(0,Lz,50)
fig,[scores,errs] = s.plot_integr("z", zz, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
tt = np.linspace(0,Lt,50)
fig,[scores,errs] = s.plot_integr("t", tt, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
zz = np.linspace(0,Lz,30)
tt = np.linspace(0,Lt,30)
fig,[scores,errs] = s.plot2D_integr(["z","t"], [zz,tt], vec0, vec1, scale="log")
plt.show()

In [ ]:
# Polar plots

vec0 = None
vec1 = None
tth = np.linspace(0,90,50)
fig,[scores,errs] = s.plot_integr("theta", tth, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
pp = np.linspace(-180,180,50)
fig,[scores,errs] = s.plot_integr("phi", pp, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
tth = np.linspace(0,90,30)
pp = np.linspace(-180,180,30)
fig,[scores,errs] = s.plot2D_integr(["mu","phi"], [tth,pp], vec0, vec1)
plt.show()

In [ ]:
# Isotrop plots

vec0 = None
vec1 = None
ddz = np.linspace(0.9,1,50)
fig,[scores,errs] = s.plot_integr("dz", ddz, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
ddy = np.linspace(-0.1,0.1,50)
fig,[scores,errs] = s.plot_integr("dy", ddy, vec0, vec1)
plt.show()

vec0 = None
vec1 = None
ddx = np.linspace(-0.1,0.1,20)
ddy = np.linspace(-0.1,0.1,20)
fig,[scores,errs] = s.plot2D_integr(["dx","dy"], [ddx,ddy], vec0, vec1)
plt.show()